In [ ]:
#| label: app:figure4

from ipywidgets import Image, Label, Layout, VBox, SelectionSlider, HBox
from IPython.display import display
from PIL import Image as Img
import numpy as np
from io import BytesIO
from cv2 import resize, imread

Image.MAX_IMAGE_PIXELS = None

def Downgrade(img):
    new_width = int(img.shape[1] * 0.5)  # Reduce width by 50%
    new_height = int(img.shape[0] * 0.5)  # Reduce height by 50%
    new_dimensions = (new_width, new_height)
    img2 = resize(img, new_dimensions)
    return img2

def imgToNpy(img, downgrade=True):
    img = imread(img, 1)[:,:,0]
    if downgrade:
        img = Downgrade(img)
    return img

def imgToBuffer(img):
    img = Img.fromarray(img)
    buffer = BytesIO()
    img.save(buffer, format='PNG')
    buffer.seek(0)
    return buffer.getvalue()

# Fixed image
files_path = "../Data/Widget1_Data.npz"
loaded_data = np.load(files_path)
fixed_image=loaded_data["raw_image_10ys"][1425:2425,1500:2500]
fixed_image_widget=Image(value=imgToBuffer(fixed_image), format='png', width=250, height=250)



# Placeholder for changing image
changing_image={"10 ys":Downgrade(loaded_data["raw_image_10ys"][1425:2425,1500:2500]),
                "1 ys":Downgrade(loaded_data["raw_image_1ys"][1425:2425,1500:2500]),
                "100 ns":Downgrade(loaded_data["raw_image_100ns"][1425:2425,1500:2500])}
changing_image2={"10 ys":Downgrade(loaded_data["final_pores_10ys"]),
                "1 ys":Downgrade(loaded_data["final_pores_1ys"]),
                "100 ns":Downgrade(loaded_data["final_pores_100ns"])}


del loaded_data

changing_image_widget = Image(value=imgToBuffer(changing_image["10 ys"]), format='png', width=250, height=250)
changing_image_widget2 = Image(value=imgToBuffer(changing_image2["10 ys"]), format='png', width=250, height=250)


# Create a label widget
label_widget_fixed = Label(value="Benchmark",layout=Layout(justify_content="center", display="flex"))
label_widget = Label(value="Dwell",layout=Layout(justify_content="center", display="flex"))
label_widget2 = Label(value="Analysed",layout=Layout(justify_content="center", display="flex"))


# Combine the label and image in a vertical box layout
labeled_image_fixed = VBox([label_widget_fixed, fixed_image_widget])
labeled_image = VBox([label_widget, changing_image_widget])
labeled_image2 = VBox([label_widget2, changing_image_widget2])


# Slider widget
slider = SelectionSlider(
    options=list(changing_image.keys()),
    value='10 ys',
    description='Dwell time:',
    continuous_update=True
)

def update_image(change):
    """Update the changing image when the slider value changes."""
    new_image=changing_image[change["new"]]
    changing_image_widget.value = imgToBuffer(new_image)


    new_image2=changing_image2[change["new"]]
    changing_image_widget2.value = imgToBuffer(new_image2)

# Observe slider changes
slider.observe(update_image, names='value')

# Centering the layout
image_box = HBox(
    [labeled_image_fixed, labeled_image,labeled_image2],
    layout=Layout(justify_content='center', align_items='center')
)
ui = VBox(
    [image_box, slider],
    layout=Layout(justify_content='center', align_items='center')
)

display(ui)